## Servicio Social – Comparativa de Modelos de Visión Computacional para Detección de Contaminantes en Cosechas de Chile 🌶️

In [2]:
import torch

print(torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # esto por si hay disponible una GPU si no usa la CPU
print("CUDA:", torch.cuda.is_available())

2.9.0+cu126
CUDA: True


In [3]:
import zipfile
import os

zip_file_path = '/content/contaminante.zip'
extract_dir = '/content/'

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"'{zip_file_path}' decompressed to '{extract_dir}'")

'/content/contaminante.zip' decompressed to '/content/'


## Integridad de los datos

In [4]:
import os

base_path = r"/content/contaminante"
data_types = ["train", "valid", "test"]

for split in data_types:
    images_path = os.path.join(base_path, split, "images")
    labels_path = os.path.join(base_path, split, "labels")

    num_images = len([
        f for f in os.listdir(images_path)
        if f.lower().endswith((".png", ".jpg", ".jpeg"))
    ])

    num_labels = len(os.listdir(labels_path))

    print(f"{split.upper()}:")
    print(f"  Imágenes: {num_images}")
    print(f"  Labels:   {num_labels}")


TRAIN:
  Imágenes: 3704
  Labels:   3704
VALID:
  Imágenes: 1236
  Labels:   1236
TEST:
  Imágenes: 1232
  Labels:   1232


### Pre-Procesamiento de las imagenes 

### Implementacion del modelo: YoloV8

In [ ]:
# =========================
# 1. IMPORTACIONES
# =========================
from ultralytics import YOLO
import torch
import os

# =========================
# 2. VERIFICAR GPU
# =========================
print("GPU disponible:", torch.cuda.is_available())

# =========================
# 3. RUTAS
# =========================
BASE_PATH = "/content/drive/MyDrive/ServicioSocial"
DATA_YAML = os.path.join(BASE_PATH, "data.yaml")

# =========================
# 4. INSTANCIAR MODELO
# =========================
model = YOLO("yolov8n.pt")  # pesos preentrenados

# =========================
# 5. ENTRENAMIENTO
# =========================
model.train(
    data=DATA_YAML,
    epochs=50,
    imgsz=640,
    batch=16,
    device=0  # usa GPU automáticamente
)

# =========================
# 6. VALIDACIÓN
# =========================
metrics = model.val()
print(metrics)

# =========================
# 7. CARGAR MEJOR MODELO
# =========================
best_model_path = "runs/detect/train/weights/best.pt"
model = YOLO(best_model_path)

# =========================
# 8. INFERENCIA
# =========================
results = model.predict(
    source=f"{BASE_PATH}/dataset/test/images",
    conf=0.25,
    save=True
)

print("Detección finalizada")


### Evaluacion del modelo